# Credit cards
simple credit card regression


In [1]:
# Get data from here:
# https://packt.link/wensZ

In [2]:
import pandas as pd

In [3]:
 df = pd.read_excel('~/Downloads/default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [4]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
# Find if there are duplicates
df['ID'].nunique()

29687

In [7]:
# Compare to the n umber of rows
df.shape

(30000, 25)

In [17]:
# Find out if we have a few accounts duplicated a lot, or a lot of individual accounts
id_counts = df['ID'].value_counts()
id_counts.head()
# id_counts.index[0:5]
# len(list(id_counts))     # 29687

2ba2d987-103e    2
98395d08-9643    2
bbffebbc-e3c4    2
f5e3478e-cf68    2
db91a87c-3cbe    2
Name: ID, dtype: int64

In [9]:
# Show distribution of duplicates
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

In [18]:
dupe_mask = id_counts >= 2
dupe_mask[0:5]

2ba2d987-103e    True
98395d08-9643    True
bbffebbc-e3c4    True
f5e3478e-cf68    True
db91a87c-3cbe    True
Name: ID, dtype: bool

In [19]:
dupe_ids = id_counts.index[dupe_mask]

In [22]:
dupe_ids = list(dupe_ids)
# Should be 30000 - 29687 = 313
len(dupe_ids)
# dupe_ids[0:5]

['2ba2d987-103e',
 '98395d08-9643',
 'bbffebbc-e3c4',
 'f5e3478e-cf68',
 'db91a87c-3cbe']

In [24]:
# Lets look at the dup rows and see what we can do with them
df.loc[df['ID'].isin(dupe_ids[0:3])]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1070,98395d08-9643,230000,1,1,1,61,-1,-1,-1,-1,...,4932,2783,7411,3710,2266,7338,2791,7433,4527,0
1170,98395d08-9643,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7383,bbffebbc-e3c4,60000,1,2,1,54,2,2,0,0,...,37613,39038,37814,55,2400,1316,2000,1531,1477,0
7483,bbffebbc-e3c4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24834,2ba2d987-103e,350000,1,1,2,30,2,2,-1,2,...,1248,832,416,0,1844,0,0,0,832,1
24934,2ba2d987-103e,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Create a boolean matrix
df_zero_mask = df == 0
df_zero_mask.shape
df_zero_mask.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,False,True,True,True,True,False
1,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,False,False
2,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True


In [31]:
# Create a mask if all fields after the ID are True, i.e., zero
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
feature_zero_mask.head()
sum(feature_zero_mask)   # Should be 313 trues in this

315

In [33]:
# since we see 315, there may be other bad as well, get rid of all of it
df_clean_1 = df.loc[~feature_zero_mask, :].copy()

In [34]:
df_clean_1.shape

(29685, 25)

In [35]:
df_clean_1['ID'].nunique()

29685

In [37]:
# Both got 29685, we've successfully eliminated dups
df_clean_1.to_csv('Data/df_clean_1.csv', index=False)